## A model can be deployed following several patterns:

    • statically, as a part of an installable software package,
    • dynamically on the user’s device,
    • dynamically on a server, or
    • via model streaming.

## 1.Static:
Depending on the operating system and the runtime environment, the objects of both the model and the feature extractor can be packaged as a part of a dynamic-link library (DLL on Windows), Shared Objects (*.so files on Linux), or be serialized and saved in the standard resource location for virtual machine-based systems, such as Java and .Net.

## 2. Dynamic Deployment on User’s Device

Dynamic deployment can be achieved in several ways:

    • by deploying model parameters,
    • by deploying a serialized object, and
    • by deploying to the browser.

### 2.1 Deployment of Model Parameters
In this deployment scenario, the **model file only contains the learned parameters**, while the user’s device has installed a runtime environment for the model. Some machine learning packages, like **TensorFlow**, have a lightweight version that can run on mobile devices.

### 2.2 Deployment of a Serialized Object
Here, the model file is a serialized object that the application would deserialize. The advantage of this approach is that you don’t need to have a runtime environment for your model on the user’s device. Like the pickle file generatin and consumed model via joblib.dump and joblib.load

### 2.3 Deploying to Browser
TensorFlow.js, have versions that allow to train and run a model in a browser, by using JavaScript as a runtime. 

It’s even possible to train a TensorFlow model in Python, and then deploy it to, and run it in the browser’s JavaScript runtime environment.

**Please look at flask_api.ipynb and predict.ipynb**


## 3 Dynamic Deployment on a Server
Because of the above complications, and problems with performance monitoring, the most frequent deployment pattern is to place the model on a server (or servers), and make it available as a Representational State Transfer application programming interface (**REST API**) in the form of a web service, or Google’s Remote Procedure Call (**gRPC**) service.

### 3.1 Deployment on a Virtual Machine
A web service running on a virtual machine receives a user request containing the input data, calls the machine learning
system on that input data, and then transforms the output of the machine learning system into the output (JSON) or (XML) string.

A load balancer dispatches the incoming requests to a specific virtual machine, depending on its availability. The virtual machines can be added and closed manually, or be a part of an auto-scaling group that launches or terminates virtual machines based on their usage.

In Python, a REST API web service is usually implemented using a web application framework such as **Flask or FastAPI**. An R equivalent is **Plumber**. TensorFlow, a popular framework used to train deep models, comes with TensorFlow Serving,
a built-in gRPC service.

Among the downsides, there is a need to maintain servers (physical or virtual). If virtualization is used, then there is an additional computational overhead due to virtualization and running multiple operating systems. Another is network latency, which can be a serious issue, depending on how fast you need to process scoring results. Finally, deploying on a virtual
machine has a relatively higher cost, compared to deployment in a container, or a serverless deployment.

### 3.2 Deployment in a Container

A container is similar to a virtual machine, in the sense that it is also an isolated runtime environment with its own filesystem, CPU, memory, and process space.

The main difference, however, is that all containers are running on the same virtual or physical machine and share the operating system, while each virtual machine runs its own instance of the operating system.

The machine learning system and the web service are installed inside a container like in a **Docker container**. Then a **container-orchestration system** is used to run the containers on a cluster of physical or virtual servers. A typical choice is **Kubernetes**. Some cloud platforms provide both their own container-orchestration engine, such as **AWS Fargate** and **Google Kubernetes Engine**, and support Kubernetes natively.

Deployment in a container has the advantage of being more resource-efficient as compared to the deployment on a virtual machine. It allows the possibility to automatically scale with scoring requests. It also allows us to scale-to-zero.

One drawback is that the containerized deployment is generally seen as more complicated, and requires expertise.


### 3.3 Serverless Deployment

Lambda-functions on AWS, and Functions on Microsoft Azure and Google Cloud Platform are typical example of serverless execution.

The serverless deployment consists of preparing a zip archive with all the code needed to run the machine learning system (model, feature extractor, and scoring code). The zip archive must contain a file with a specific name that contains a specific function, or class-method definition with a specific signature (an entry point function). The zip archive is uploaded to
the cloud platform and registered under a unique name.

The cloud platform provides an API to submit inputs to the serverless function. This specifies its name, provides the payload, and yields the outputs. The cloud platform takes care of deploying the code and the model on an adequate computational resource, executing the code, and routing the output back to the client.

The zip file size limit can be a challenge. A typical machine learning model requires multiple heavyweight dependencies. Python’s libraries, to include Numpy, SciPy, and scikit-learn, are often needed for the model to be properly executed.

There are advantages, cost-efficient: you only pay for compute-time, you don’t have to provision resources such as servers or virtual machines. You don’t have to install dependencies, maintain, or upgrade the system. Serverless systems are highly
scalable and can easily and effortlessly support thousands of requests per second. Serverless functions support both synchronous and asynchronous modes of operation.

Serverless deployment also simplifies **canary deployment, or canarying**. In software engineering, **canarying is a strategy when the updated code is pushed to just a small group of end-users**, usually unaware. Because the new version is only distributed to a small number of users, **its impact is relatively low, and changes can be reversed quickly**, should the new
code contain bugs. It is easy to set up two versions of serverless functions in production, and start sending low volume traffic to just one, and test it without affecting many users.

## Caching
The first time the function run_model is called for some input, model.predict will be called.

For the subsequent calls of run_model with the same value of the input, the output will be read 
from cache that memorizes the result of maxsize most recent calls of model.predict.

In [ ]:
from functools import lru_cache

# Read the model from file
model = pickle.load(open("model_file.pkl", "rb"))

@lru_cache(maxsize=500)
def run_model(input_example):
    return model.predict(input_example)

# Now you can call run_model
# on new data


### MLOps must be language-, framework-, platform-, and infrastructure-agnostic practice. MLOps should follow a “convention over configuration” implementation.

**The MLOps technology stack should include tooling for the following tasks:**

1. data engineering,
2. version control of data, ML models and code,
3. coninuous integration and continuous delivery pipelines,
4. automating deployments and experiments,
5. model performance assessment, and
6. model monitioring in production.